In [ ]:
import requests
from urllib.parse import urlencode
from datetime import date
import json
import datetime
import difflib
import sqlite3
import os
import pandas as pd
import numpy as np
from src.credentials import epias_uname, epias_pwd

def plant_investments(tgt):
    licensed_powerplant_investment_list = 'https://seffaflik.epias.com.tr/electricity-service/v1/generation/data/licensed-powerplant-investment-list'
    today = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S+03:00")
    
    request_json = {
        "startDate": "2021-01-01T00:00:00+03:00",
        "endDate": today,
    }
    
    headers = {'TGT': f'{tgt}'}
    licensed_powerplant_investment_list = requests.post(licensed_powerplant_investment_list, json=request_json, headers=headers)
    
    with open('licensed_powerplant_investment_list.json', "w") as f:
        f.write(json.dumps(json.loads(licensed_powerplant_investment_list.text), indent=4, sort_keys=True, ensure_ascii=False))
        
def region_list(tgt):
    region = 'https://seffaflik.epias.com.tr/electricity-service/v1/generation/data/region-list'
    
    headers = {'TGT': f'{tgt}'}
    region = requests.get(region, headers=headers)
    
    with open('region.json', "w") as f:
        f.write(json.dumps(json.loads(region.text), indent=4, sort_keys=True, ensure_ascii=False))
    
    return region

def org_list(tgt):
    organizations = 'https://seffaflik.epias.com.tr/electricity-service/v1/generation/data/organization-list'
    today = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S+03:00")
    
    request_json = {
        "startDate": "2024-09-04T00:00:00+03:00",
        "endDate": today,
    }
    
    headers = {'TGT': f'{tgt}'}
    organizations = requests.post(organizations, json=request_json, headers=headers)
    
    with open('organizations.json', "w") as f:
        f.write(json.dumps(json.loads(organizations.text), indent=4, sort_keys=True, ensure_ascii=False))
        
    return organizations

def uevm_plant_list(tgt):
    uevm_plant_list = 'https://seffaflik.epias.com.tr/electricity-service/v1/generation/data/injection-quantity-powerplant-list'
    
    headers = {'TGT': f'{tgt}'}
    uevm_plant_list = requests.get(uevm_plant_list, headers=headers)
    
    with open('uevm_plant_list.json', "w") as f:
        f.write(json.dumps(json.loads(uevm_plant_list.text), indent=4, sort_keys=True, ensure_ascii=False))
    
    return uevm_plant_list

def plant_list(tgt):
    santral_listesi = 'https://seffaflik.epias.com.tr/electricity-service/v1/generation/data/powerplant-list'
    
    headers = {'TGT': f'{tgt}'}
    plant_list = requests.get(santral_listesi, headers=headers)
    
    with open('plant_list.json', "w") as f:
        f.write(json.dumps(json.loads(plant_list.text), indent=4, sort_keys=True, ensure_ascii=False))
    
    return plant_list

In [ ]:
# TGT
tgt_url = 'https://giris.epias.com.tr/cas/v1/tickets'

request_data = {
    "username": epias_uname,
    "password": epias_pwd
}
encoded_data = urlencode(request_data)

headers = {'Accept': 'text/plain', 'Content-Type': 'application/x-www-form-urlencoded'}
tgt_response = requests.post(tgt_url, data=encoded_data, headers=headers)

if tgt_response.status_code != 201:
    print(f"Failed to get TGT. Status code: {tgt_response.status_code}")
    print(tgt_response.text)  # Print any error message from the server

In [ ]:
# convert to dataframe and write to the database
plants = pd.DataFrame(json.loads(plant_list(tgt_response.text).text).get('items', []))
uevm_plants = pd.DataFrame(json.loads(uevm_plant_list(tgt_response.text).text).get('items', []))
organizations = pd.DataFrame(json.loads(org_list(tgt_response.text).text).get('items', []))

In [ ]:
plants_df = plants.copy()
uevm_plants_df = uevm_plants.copy()
organizations_df = organizations.copy()

path = os.path.abspath('/home/kutlay/WFD/wfd.db')
#path = os.path.abspath('/home/wfd/wfd_izmir.db') # remote server

conn = sqlite3.connect(path)

# get wf table
query = "SELECT * FROM wf"
wf = pd.read_sql_query(query, conn)
wf_initial = wf.copy()

def match_name(name, info_df):
    return difflib.get_close_matches(name, info_df['name_match'], n=1, cutoff=0.99)

In [ ]:
wf_match = wf.copy()
wf_match = wf_match.dropna(subset=['plant_name'])
wf_match = wf_match[wf_match['epias_info_lock'] == 0]
wf_match = wf_match[wf_match['license_status'] == 'Yürürlükte']

plants_df['name_match'] = plants_df['name'].str[:-17]
uevm_plants_df['name_match'] = uevm_plants_df['name'].str[:-17]

matched_names = wf_match['plant_name'].apply(lambda x: match_name(x, plants_df))
matched_names = matched_names.apply(lambda x: x[0] if x else None)

uevm_matched_names = wf_match['plant_name'].apply(lambda x: match_name(x, uevm_plants_df))
uevm_matched_names = uevm_matched_names.apply(lambda x: x[0] if x else None)

print(f"Unmatched names in plants: {matched_names.isna().sum()}")
print(f"Unmatched names in uevm_plants: {uevm_matched_names.isna().sum()}")

In [ ]:
for index, name_match in matched_names.items():
    if name_match:
        plant = plants_df[plants_df['name_match'] == name_match]
        eic = plant['eic'].values[0]
        shortName = plant['shortName'].values[0]
        id = plant['id'].values[0].astype(np.int64).item()
        name = plant['name'].values[0]
        wf.loc[index, 'epias_plant_id'] = id
        wf.loc[index, 'epias_plant_name'] = name
        wf.loc[index, 'epias_plant_eic'] = eic
        wf.loc[index, 'epias_plant_shortname'] = shortName
        wf.loc[index, 'epias_region'] = 'TR1' # region for every plant is TR1 on EPİAŞ
        
for index, name_match in uevm_matched_names.items():
    if name_match:
        plant = uevm_plants_df[uevm_plants_df['name_match'] == name_match]
        eic = plant['eic'].values[0]
        shortName = plant['shortName'].values[0]
        id = plant['id'].values[0].astype(np.int64).item()
        name = plant['name'].values[0]
        wf.loc[index, 'epias_uevm_plant_id'] = id
        wf.loc[index, 'epias_uevm_plant_name'] = name
        wf.loc[index, 'epias_uevm_plant_eic'] = eic
        wf.loc[index, 'epias_uevm_plant_shortname'] = shortName

In [ ]:
wf_match = wf.copy()
wf_match = wf_match.dropna(subset=['license_holder'])
wf_match = wf_match[wf_match['epias_info_lock'] == 0]
wf_match = wf_match[wf_match['license_status'] == 'Yürürlükte']

wf_match['license_holder'] = wf_match['license_holder'].astype(str).str.split().str[:2].str.join(' ')
organizations_df['name_match'] = organizations_df['organizationName'].astype(str).str.split().str[:2].str.join(' ')


matched_names = wf_match['license_holder'].apply(lambda x: match_name(x, organizations_df))
matched_names = matched_names.apply(lambda x: x[0] if x else None)


print(f"Unmatched names in organizations: {matched_names.isna().sum()}")

In [ ]:
for index, name_match in matched_names.items():
    if name_match:
        org = organizations_df[organizations_df['name_match'] == name_match]
        id = org['organizationId'].values[0].astype(np.int64).item()
        name = org['organizationName'].values[0]
        etso = org['organizationEtsoCode'].values[0]
        shortName = org['organizationShortName'].values[0]
        wf.loc[index, 'epias_organization_id'] = id
        wf.loc[index, 'epias_organization_name'] = name
        wf.loc[index, 'epias_organization_etso_code'] = etso
        wf.loc[index, 'epias_organization_shortname'] = shortName

In [ ]:
gate = input("Do you know what you are doing? (y/n): ")
if gate == 'y':
    gate_2 = input("Are you sure??? (y/n): ")
    if gate_2 == 'y':
        wf.to_sql('wf', conn, if_exists='replace', index=False)
        print("congrats")
    else:
        print("Database update cancelled.")
else:
    print("Database update cancelled.")

In [ ]:
# compare the initial and updated dataframes to see the changes
wf_updated = wf.copy()
wf_updated = wf_updated.dropna(subset=['plant_name'])
wf_updated = wf_updated[wf_updated['epias_info_lock'] == 0]

wf_initial = wf_initial.dropna(subset=['plant_name'])
wf_initial = wf_initial[wf_initial['epias_info_lock'] == 0]

wf_initial = wf_initial[['plant_name', 'epias_plant_id', 'epias_plant_name', 'epias_plant_eic', 'epias_plant_shortname', 'epias_region', 'epias_uevm_plant_id', 'epias_uevm_plant_name', 'epias_uevm_plant_eic', 'epias_uevm_plant_shortname', 'epias_organization_id', 'epias_organization_name', 'epias_organization_etso_code', 'epias_organization_shortname']]
wf_updated = wf_updated[['plant_name', 'epias_plant_id', 'epias_plant_name', 'epias_plant_eic', 'epias_plant_shortname', 'epias_region', 'epias_uevm_plant_id', 'epias_uevm_plant_name', 'epias_uevm_plant_eic', 'epias_uevm_plant_shortname', 'epias_organization_id', 'epias_organization_name', 'epias_organization_etso_code', 'epias_organization_shortname']]

wf_diff = pd.concat([wf_initial, wf_updated]).drop_duplicates(keep=False)

In [ ]:
wf_diff